In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train=pd.read_csv(r'C:\Python\kaggle\@ON-Favorita货物销量预测\data\train_2017.csv',parse_dates=["date"],
                  usecols=[1, 2, 3, 4, 5],dtype={'onpromotion': bool})
test = pd.read_csv(r'C:\Python\kaggle\@ON-Favorita货物销量预测\data\test.csv',usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},parse_dates=["date"])

In [14]:
#store_nbr	item_nbr	records_number	windows_records_number     date	store_nbr	item_nbr	unit_sales	onpromotion
store_all=train['store_nbr'].unique()
item_all=train['item_nbr'].unique()
date_all=train['date'].unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)

train=train.reindex(
    pd.MultiIndex.from_product((store_all,item_all,date_all),names=['store_nbr','item_nbr','date'])
)

In [15]:
train_sales=train[['unit_sales']].unstack(level=-1)
train_promo=train[['onpromotion']].unstack(level=-1)

In [18]:
train_sales

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            NaN         NaN         NaN         NaN   
          99197            NaN         NaN         NaN         NaN   
          103501           NaN         NaN         NaN         NaN   
          103520           NaN         NaN         NaN         NaN   
          103665           NaN         NaN         NaN         NaN   
          105574           NaN         NaN         NaN         NaN   
          105575           NaN         NaN         NaN         NaN   
          105576           NaN         NaN         NaN         NaN   
          105577           NaN         NaN         NaN         NaN   
          105693           NaN         NaN         NaN         NaN   
          105737           NaN         NaN         NaN         NaN   
          105857           NaN         NaN         NaN         NaN   
          106716           NaN         NaN         NaN         NaN   
          108079           NaN         NaN         NaN         NaN   
          108634           NaN         NaN         NaN         NaN   
          108696           NaN         NaN         NaN         NaN   
          108698           NaN         NaN         NaN         NaN   
          108701           NaN         NaN         NaN         NaN   
          108786           NaN         NaN         NaN         NaN   
          108797           NaN         NaN         NaN         NaN   
          108831           NaN         NaN         NaN         NaN   
          108833           NaN         NaN         NaN         NaN   
          108862           NaN         NaN         NaN         NaN   
          108952           NaN         NaN         NaN         NaN   
          111223           NaN         NaN         NaN         NaN   
          111397           NaN         NaN         NaN         NaN   
          112830           NaN         NaN         NaN         NaN   
          114778           NaN         NaN         NaN         NaN   
          114790           NaN         NaN         NaN         NaN   
          114799           NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
54        2116238          NaN         NaN         NaN         NaN   
          2116416          NaN         NaN         NaN         NaN   
          2118662          NaN         NaN         NaN         NaN   
          2120723          NaN         NaN         NaN         NaN   
          2121610          NaN         NaN         NaN         NaN   
          2121690          NaN         NaN         NaN         NaN   
          2122188          NaN         NaN         NaN         NaN   
          2122676          NaN         NaN         NaN         NaN   
          2122818          NaN         NaN         NaN         NaN   
          2122868          NaN         NaN         NaN         NaN   
          2122947          NaN         NaN         NaN         NaN   
          2123036          NaN         NaN         NaN         NaN   
          2123209          NaN         NaN         NaN         NaN   
          2123410          NaN         NaN         NaN         NaN   
          2123463          NaN         NaN         NaN         NaN   
          2123468          NaN         NaN         NaN         NaN   
          2123711          NaN         NaN         NaN         NaN   
          2123727          NaN         NaN         NaN         NaN   
          2123747          NaN         NaN         NaN         NaN   
          2123750          NaN         NaN         NaN         NaN   
          2123775          NaN         NaN         NaN         NaN   
          2123790          NaN         NaN         NaN         NaN   
          2123791          NaN         NaN         NaN         NaN   
          2123839          NaN         NaN         NaN         NaN   
          

In [19]:
train_promo.shape

(216972, 227)

In [17]:
train_sales.columns=train_sales.columns.get_level_values(1)
train_promo.columns=train_promo.columns.get_level_values(1)

In [ ]:
td=timedelta(days=15)
timespan=[
            (pd.datetime(2017,8,15)-td,pd.datetime(2017,8,15)),
            (pd.datetime(2017,7,16),pd.datetime(2017,7,16)+td),
            (pd.datetime(2017,7,15)-td,pd.datetime(2017,7,15)),
            (pd.datetime(2017,6,16),pd.datetime(2017,6,16)+td),
            (pd.datetime(2017,6,15)-td,pd.datetime(2017,6,15)),
            (pd.datetime(2017,5,16),pd.datetime(2017,5,16)+td),
            (pd.datetime(2017,5,15)-td,pd.datetime(2017,5,15)),
            (pd.datetime(2017,4,16),pd.datetime(2017,4,16)+td),
            (pd.datetime(2017,4,15)-td,pd.datetime(2017,4,15)),
            (pd.datetime(2017,3,16),pd.datetime(2017,3,16)+td),
           ]

def prepare_dataset(dataset_number,promo=True,y=True):
    '''
    从原始训练集返回训练集的Y和特征窗口Z，顺序分别为：Y,Z,Y_promo,Z_promo
    '''

    train_tmp=train[(train['date']>=timespan[dataset_number][0]) & (train['date']<=timespan[dataset_number][1])].cpoy()
    
    #样本组合采样
    train_tmp_store=train_tmp['store_nbr'].unique()
    train_tmp_item=train_tmp['item_nbr'].unique()
    train_tmp.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
    train_tmp=train_tmp.reindex(
        pd.MultiIndex.from_product((train_tmp_store,train_tmp_item,date_all),names=['store_nbr','item_nbr','date'])
    )
    
    #采样样本全时间
    train_tmp_sales=train[['unit_sales']].unstack(level=-1)
    train_tmp_promo=train[['onpromotion']].unstack(level=-1)
    train_tmp_sales.columns=train_tmp_sales.columns.get_level_values(1)
    train_tmp_promo.columns=train_tmp_promo.columns.get_level_values(1)
    
    #采样样本的Y
    Y=train_tmp_sales[pd.date_range(start=timespan[dataset_number][0],end=timespan[dataset_number][1],freq='D')]
    Y_promo=train_tmp_promo[pd.date_range(start=timespan[dataset_number][0],end=timespan[dataset_number][1],freq='D')]
    
    if promo=True:
        if y=True:
            return Y,train_tmp_sales,Y_promo,train_tmp_promo
        if y=False:
            return train_tmp_sales,train_tmp_promo
    if promo=False:
        if y=True:
            return Y,train_tmp_sales
        if y=False:
            return train_tmp_sales

In [ ]:
feature_windows=[3,5,7,15,30,60]
def get_feature(dataset_number，feature_windows=feature_windows):
    '''
    通过条目-2017时间的销量构造【历史销量】相关统计量
    '''
    train_tmp_sales=prepare_dataset(dataset_number,promo=False,y=False)
    feature=pd.DataFrame(index=train_tmp_sales.index) 
    for i in feature_windows:
        train_tmp_sales_in_windows=train_tmp_sales[pd.date_range(end=timespan[dataset_number][0]-timedelta(days=1),periods=i,freq='D')]
        
        #空值个数，记录个数
        feature['na_count_{}'.format(i)]=train_tmp_sales_i.isnull().sum(axis=1).values
        feature['record_count_{}'.format(i)]=i-train_tmp_sales_i['na_count_{}'.format(i)]
        
        #是否全空，是否全满
        feature['is_all_na_{}'.format(i)]=True if feature['record_count_{}'.format(i)]==0 else False
        feature['is_all_full_{}'.format(i)]=True if feature['na_count_{}'.format(i)]==0 else False
        
        #累加，有记录的平均值
        feature['cumsum_{}'.format(i)]=train_tmp_sales_i.cumsum(axis=1).values
        feature['means_{}'.format(i)]=feature['cumsum_{}'.format(i)]/feature['record_count_{}'.format(i)]
        
        #有记录的中位数，有记录的方差
        feature['median_{}'.format(i)]=train_tmp_sales_i.median(axis=1).values
        feature['var_{}'.format(i)]=train_tmp_sales_i.var(axis=1).values
        
        #最大纪录当天的占比
        feature['max_{}'.format(i)]=train_tmp_sales_i.max(axis=1).values
        feature['max_cumsum_ratio_{}'.format(i)]=feature['max_{}'.format(i)]/feature['cumsum_{}'.format(i)]
        
    #不同视窗累积销量占比   
    for i in [(5,10),(7,15),(15,30)]:
        feature['cumsum_ratio_{0}_{1}'.format(i[0],i[1])]=feature['cumsum_{}'.format(i[0])]/feature['cumsum_{}'.format(i[1])]
        
    return feature

In [ ]:
def get_feature_

In [ ]:
items=pd.read_csv(r'C:\Python\kaggle\@ON-Favorita货物销量预测\data\items.csv')

#构造关于items的特征
def get_feature_perishable():
    '''
    加入腐烂特征
    '''
    

In [ ]:
def splite_train_cv(cv=1):
    '''
    构造 train_x,cv_x
    '''
    if cv==1:
        cv_x=get_feature(1)
        
        dataset_number=range(3,11)
        train_x=get_feature(2)
        for i in train_dataset_number:
            train_x=pd.concat(train_x,get_feature(i))
    else:
        cv_x=get_feature(cv)
        
        dataset_number=range(1,11)
        train_x=get_feature(1)
        for i in train_dataset_number:
            if i!=cv:
                train_x=pd.concat(train_x,get_feature(i))
        
    return train_x,cv_x

In [ ]:
dic={}
name=locals()


    
    name['train_%s' % i]=
    dic[i]='train_%s' % i

In [32]:
train_1

date                2017-07-31  2017-08-01  2017-08-02  2017-08-03  \
store_nbr item_nbr                                                   
1         96995            NaN         NaN         NaN         NaN   
          99197            NaN         NaN         NaN         NaN   
          103501           NaN         NaN         NaN         NaN   
          103520           NaN         NaN         NaN         NaN   
          103665           NaN         NaN         NaN         NaN   
          105574           NaN         NaN         NaN         NaN   
          105575           NaN         NaN         NaN         NaN   
          105576           NaN         NaN         NaN         NaN   
          105577           NaN         NaN         NaN         NaN   
          105693           NaN         NaN         NaN         NaN   
          105737           NaN         NaN         NaN         NaN   
          105857           NaN         NaN         NaN         NaN   
          106716           NaN         NaN         NaN         NaN   
          108079           NaN         NaN         NaN         NaN   
          108634           NaN         NaN         NaN         NaN   
          108696           NaN         NaN         NaN         NaN   
          108698           NaN         NaN         NaN         NaN   
          108701           NaN         NaN         NaN         NaN   
          108786           NaN         NaN         NaN         NaN   
          108797           NaN         NaN         NaN         NaN   
          108831           NaN         NaN         NaN         NaN   
          108833           NaN         NaN         NaN         NaN   
          108862           NaN         NaN         NaN         NaN   
          108952           NaN         NaN         NaN         NaN   
          111223           NaN         NaN         NaN         NaN   
          111397           NaN         NaN         NaN         NaN   
          112830           NaN         NaN         NaN         NaN   
          114778           NaN         NaN         NaN         NaN   
          114790           NaN         NaN         NaN         NaN   
          114799           NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
54        2116238          NaN         NaN         NaN         NaN   
          2116416          NaN         NaN         NaN         NaN   
          2118662          NaN         NaN         NaN         NaN   
          2120723          NaN         NaN         NaN         NaN   
          2121610          NaN         NaN         NaN         NaN   
          2121690          NaN         NaN         NaN         NaN   
          2122188          NaN         NaN         NaN         NaN   
          2122676          NaN         NaN         NaN         NaN   
          2122818          NaN         NaN         NaN         NaN   
          2122868          NaN         NaN         NaN         NaN   
          2122947          NaN         NaN         NaN         NaN   
          2123036          NaN         NaN         NaN         NaN   
          2123209          NaN         NaN         NaN         NaN   
          2123410          NaN         NaN         NaN         NaN   
          2123463          NaN         NaN         NaN         NaN   
          2123468          NaN         NaN         NaN         NaN   
          2123711          NaN         NaN         NaN         NaN   
          2123727          NaN         NaN         NaN         NaN   
          2123747          NaN         NaN         NaN         NaN   
          2123750          NaN         NaN         NaN         NaN   
          2123775          NaN         NaN         NaN         NaN   
          2123790          NaN         NaN         NaN         NaN   
          2123791          NaN         NaN         NaN         NaN   
          2123839          NaN         NaN         NaN         NaN   
          

In [ ]:
def prepare_dataset(t2017, is_train=True):